CC5213 - Recuperación de Información Multimedia
==
Profesor: Juan Manuel Barrios  
Fecha: 26 abril 2021

Comparación búsquedas aproximadas k-d tree y k-means tree
==

### Leer los datos

In [ ]:
import os
import numpy

def load_file(filename, dimensions):
    assert os.path.isfile(filename), "no existe archivo " + filename
    mat = numpy.fromfile(filename, dtype=numpy.float32)
    return numpy.reshape(mat, (-1, dimensions))

class Dataset:   
    def __init__(self, nombre, archivo_q, archivo_r, dimensiones):
        self.nombre = nombre
        self.q = load_file(archivo_q, dimensiones)
        self.r = load_file(archivo_r, dimensiones)
        print("Dataset {}: Q={} R={}".format(self.nombre, self.q.shape, self.r.shape))

dataset1 = Dataset("SIFT", "SIFT-Q-128_float32.bin", "SIFT-R-128_float32.bin", 128)

dataset2 = Dataset("MEL", "MEL-Q-128_float32.bin", "MEL-R-128_float32.bin", 128)


### Objeto que hace la comparación

In [ ]:
import time
import pyflann

class Curva:   
    def __init__(self):
        self.precisiones = []
        self.eficiencias = []

class EvaluarArbol:   
    def __init__(self, dataset):
        self.dataset = dataset
        self.flann = pyflann.FLANN()
        
    def linear_scan(self):
        print("linear scan {} de Q={} x R={}".format(
            self.dataset.nombre, self.dataset.q.shape, self.dataset.r.shape))
        self.flann.build_index(self.dataset.r, algorithm="linear")
        t0 = time.time()
        self.gt_nns, self.gt_dists = self.flann.nn_index(self.dataset.q, num_neighbors=1, cores=4)
        self.gt_segundos = time.time() - t0
        print("linear scan = {:.1f} segundos".format(self.gt_segundos))
    
    def evaluar_busqueda(self, nns, dists, tiempo):
        correctas = 0
        incorrectas = 0
        for i in range(len(self.gt_nns)):
            if nns[i] == self.gt_nns[i] or dists[i] == self.gt_dists[i]: 
                correctas += 1
            else:
                incorrectas += 1 
        precision = 100 * correctas / (correctas + incorrectas)
        eficiencia = 100 * tiempo / self.gt_segundos
        return precision, eficiencia

    def evaluar_arbol(self, algorithm, trees=0, branching=0):
        t0 = time.time()
        self.flann.build_index(self.dataset.r, algorithm=algorithm, trees=trees, branching=branching)
        name = "{}-{}".format(algorithm, max(trees, branching))
        print("{} construcción {} = {:.2f} segundos".format(self.dataset.nombre, name, time.time() - t0))
        curva = Curva()
        for checks in (1, 10, 40, 70, 100, 200, 300, 400, 500, 600, 700, 800, 900, 1000, 5000):
            t0 = time.time()
            nn, dist = self.flann.nn_index(self.dataset.q, num_neighbors=1, cores=4, checks=checks)
            precision, eficiencia = self.evaluar_busqueda(nn, dist, time.time() - t0)
            print("  {}\tprecision={:.1f}%\ttiempo={:.1f}% checks={}".format(name, precision, eficiencia, checks))
            curva.precisiones.append(precision)
            curva.eficiencias.append(eficiencia)
        return curva


In [ ]:
ev = EvaluarArbol(dataset1)
ev.linear_scan()
curva_kd1  = ev.evaluar_arbol(algorithm="kdtree", trees=1)
curva_kd10 = ev.evaluar_arbol(algorithm="kdtree", trees=10)
curva_kd30 = ev.evaluar_arbol(algorithm="kdtree", trees=30)
curva_km3  = ev.evaluar_arbol(algorithm="kmeans", branching=3)
curva_km30 = ev.evaluar_arbol(algorithm="kmeans", branching=30)
curva_km90 = ev.evaluar_arbol(algorithm="kmeans", branching=90)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib notebook
%matplotlib inline

plt.plot(curva_kd1.precisiones,  curva_kd1.eficiencias,  label='kdtree-1',  color='r', linestyle='--', marker='o', markerfacecolor='c', markersize=8)
plt.plot(curva_kd10.precisiones, curva_kd10.eficiencias, label='kdtree-10', color='g', linestyle='--', marker='o', markerfacecolor='m', markersize=8)
plt.plot(curva_kd30.precisiones, curva_kd30.eficiencias, label='kdtree-30', color='b', linestyle='--', marker='o', markerfacecolor='y', markersize=8)
plt.plot(curva_km3.precisiones,  curva_km3.eficiencias,  label='kmeans-3',  color='c', linestyle='-.', marker='^', markerfacecolor='r', markersize=8)
plt.plot(curva_km30.precisiones, curva_km30.eficiencias, label='kmeans-30', color='m', linestyle='-.', marker='^', markerfacecolor='g', markersize=8)
plt.plot(curva_km90.precisiones, curva_km90.eficiencias, label='kmeans-90', color='y', linestyle='-.', marker='^', markerfacecolor='b', markersize=8)
plt.title(ev.dataset.nombre)
plt.xlabel('Precisión NN')
plt.ylabel('Tiempos')
plt.xlim(0,100)
plt.ylim(0,100)
plt.legend()
plt.show()


In [ ]:
ev2 = EvaluarArbol(dataset2)
ev2.linear_scan()
curva2_kd1  = ev2.evaluar_arbol(algorithm="kdtree", trees=1)
curva2_kd10 = ev2.evaluar_arbol(algorithm="kdtree", trees=10)
curva2_kd30 = ev2.evaluar_arbol(algorithm="kdtree", trees=30)
curva2_km3  = ev2.evaluar_arbol(algorithm="kmeans", branching=3)
curva2_km30 = ev2.evaluar_arbol(algorithm="kmeans", branching=30)
curva2_km90 = ev2.evaluar_arbol(algorithm="kmeans", branching=90)


In [ ]:
plt.plot(curva2_kd1.precisiones,  curva2_kd1.eficiencias,  label='kdtree-1',  color='r', linestyle='--', marker='o', markerfacecolor='c', markersize=8)
plt.plot(curva2_kd10.precisiones, curva2_kd10.eficiencias, label='kdtree-10', color='g', linestyle='--', marker='o', markerfacecolor='m', markersize=8)
plt.plot(curva2_kd30.precisiones, curva2_kd30.eficiencias, label='kdtree-30', color='b', linestyle='--', marker='o', markerfacecolor='y', markersize=8)
plt.plot(curva2_km3.precisiones,  curva2_km3.eficiencias,  label='kmeans-3',  color='c', linestyle='-.', marker='^', markerfacecolor='r', markersize=8)
plt.plot(curva2_km30.precisiones, curva2_km30.eficiencias, label='kmeans-30', color='m', linestyle='-.', marker='^', markerfacecolor='g', markersize=8)
plt.plot(curva2_km90.precisiones, curva2_km90.eficiencias, label='kmeans-90', color='y', linestyle='-.', marker='^', markerfacecolor='b', markersize=8)
plt.title(ev2.dataset.nombre)
plt.xlabel('Precisión NN')
plt.ylabel('Tiempos')
plt.xlim(0,100)
plt.ylim(0,100)
plt.legend()
plt.show()